In [1]:
import math
import os
from pathlib import Path

## Preparación del corpus de emails

In [2]:
if not os.path.isdir('datasets'):
    print ("Directory not exist")
    !git clone https://github.com/pachocamacho1990/datasets
else:
    print ("Directory exist")

Directory exist


In [3]:
if not os.path.isdir('corpus1/spam'):
    print ("Directory not exist")
    ! unzip datasets/email/plaintext/corpus1.zip
else:
    print ("Directory exist")

Directory exist


In [4]:
os.listdir('corpus1/spam')

['4470.2005-05-10.GP.spam.txt',
 '3291.2004-12-25.GP.spam.txt',
 '5128.2005-09-02.GP.spam.txt',
 '1295.2004-06-06.GP.spam.txt',
 '2285.2004-09-26.GP.spam.txt',
 '2764.2004-11-09.GP.spam.txt',
 '5011.2005-08-14.GP.spam.txt',
 '4680.2005-06-11.GP.spam.txt',
 '2218.2004-09-20.GP.spam.txt',
 '4224.2005-04-10.GP.spam.txt',
 '4746.2005-06-26.GP.spam.txt',
 '0788.2004-04-06.GP.spam.txt',
 '4328.2005-04-21.GP.spam.txt',
 '3845.2005-02-15.GP.spam.txt',
 '0899.2004-04-23.GP.spam.txt',
 '1190.2004-05-22.GP.spam.txt',
 '3959.2005-03-02.GP.spam.txt',
 '3994.2005-03-07.GP.spam.txt',
 '2755.2004-11-08.GP.spam.txt',
 '5007.2005-08-14.GP.spam.txt',
 '1471.2004-07-03.GP.spam.txt',
 '2530.2004-10-18.GP.spam.txt',
 '0596.2004-03-05.GP.spam.txt',
 '1183.2004-05-21.GP.spam.txt',
 '1735.2004-07-30.GP.spam.txt',
 '5089.2005-08-26.GP.spam.txt',
 '0367.2004-02-04.GP.spam.txt',
 '4093.2005-03-17.GP.spam.txt',
 '3183.2004-12-15.GP.spam.txt',
 '0682.2004-03-19.GP.spam.txt',
 '1402.2004-06-22.GP.spam.txt',
 '4102.2

In [5]:
data = []
clases = []
#lectura de spam data
for file in os.listdir('corpus1/spam'):
    with open('corpus1/spam/'+file, encoding='latin-1') as f:
        data.append(f.read())
        clases.append('spam')
#lectura de ham data
for file in os.listdir('corpus1/ham'):
    with open('corpus1/ham/'+file, encoding='latin-1') as f:
        data.append(f.read())
        clases.append('ham')
len(data)

5172

## Construcción de modelo Naive Bayes

### Tokenizador de Spacy

* Documentación: https://spacy.io/api/tokenizer
* ¿Cómo funciona el tokenizador? https://spacy.io/usage/linguistic-features#how-tokenizer-works

In [19]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = Tokenizer(nlp.vocab)

Aplicamos el tokenizador para nuestro texto obteneindo una lista

In [18]:
print([t.text for t in tokenizer(data[0])])

['Subject:', 'long', 'time', '\n', 'customer', 'id', ':', '615', '\n', 'order', '#', '13', '\n', 'we', 'would', 'like', 'to', 'thank', 'you', 'for', 'your', 'recent', 'purchase', 'on', 'our', 'website', '\n', 'and', 'thought', 'that', 'you', 'may', 'also', 'be', 'interested', 'in', 'our', 'new', 'lub', '.', 'ricant', '\n', 'that', 'has', 'been', 'added', 'to', 'our', 'line', 'of', 'herb', '-', 'al', 'supp', '-', 'lements', '.', '\n', 'it', 'is', 'gua', 'ran', '-', 'teed', 'to', 'help', 'spice', 'up', 'your', 'night', 'life', 'in', 'the', 'bed', '`', 'room', '.', '\n', 'if', 'your', 'not', 'completely', 'satis', 'fied', 'then', 'just', 'return', 'it', 'back', 'to', 'us', 'with', '\n', 'no', 'questions', 'asked', '.', '\n', 'you', 'can', 'view', 'our', 'new', 'line', 'at', 'http', ':', '/', '/', 'nikko', '.', 'getchatimenow', '.', 'com', '/', 'x', '/', 'poppi', '\n', 'thank', 'you', 'once', 'again', '\n']


### Clase principal para el algoritmo

Recuerda que la clase más probable viene dada por (en espacio de cómputo logarítmico): 


$$\hat{c} = {\arg \max}_{(c)}\log{P(c)}
 +\sum_{i=1}^n
\log{ P(f_i \vert c)}
$$

Donde, para evitar casos atípicos (Hay escenarios donde la probabilidad es cero, ya que la palabra no esta en el corpus), usaremos el suavizado de Laplace así:

$$
P(f_i \vert c) = \frac{C(f_i, c)+1}{C(c) + \vert V \vert}
$$

siendo $\vert V \vert$ la longitud del vocabulario de nuestro conjunto de entrenamiento. 

In [ ]:
import numpy as np

# Ceamos una clase de Naive Bayes para clasificar
class NaiveBayesClassifier():
    # Instancias propias de la clase
    nlp = English()
    tokenizer = Tokenizer(nlp.vocab) 

    # Función que corresponde al tokenizador
    # Renorna el texto completamente tokenizado en una lista
    def tokenize(self, doc):
        return [t.text.lower() for t in tokenizer(doc)]

    # Contador de palabras, ya que para calcular
    #l as probabilidades necesito contar las palabras
    def word_counts(self, words):
        wordCount = {} # Diccionario
        
        # Recorro todas las palabras
        for w in words: 
            # Verifica si la palabra esta en el diccionario
            if w in wordCount.keys():
                wordCount[w] += 1
            else:
                wordCount[w] = 1
        return wordCount

    # Metodo 'fit' donde se entrena el modelo y 
    # debe adminir como entrada los datos de entrenamiento
    def fit(self, data, clases):
        # Define la longitud de los datos de entrenamiento
        n = len(data)
        
        # Atributo que sefine cuantas clases unicas hay
        # y debe ser capaz de hacer clasificación multiclase
        self.unique_clases = set(clases)
        
        # Atributo vacio para asignación de vocabulario
        self.vocab = set()
        
        # Atributo para contar cuantas veces aparece cada 
        # categoría posible en el corpus 'C(c)'.
        self.classCount = {} #C(c)
        
        # Atributo para calcular el logaritmo de la probabilidad de la categoría 'P(c)'.
        self.log_classPriorProb = {} #P(c)
        
        # Atributo para calcular los contenos de que dada una categoría se observe una 
        # palabra en algún documento de dicha categoría 'C(w|c)'.
        self.wordConditionalCounts = {} #C(w|c)
    
        # Conteos de categorías
        for c in clases:
            if c in self.classCount.keys():
                self.classCount[c] += 1
            else:
                self.classCount[c] = 1
        # Calculo de P(c)
        for c in self.classCount.keys():
            # Por cada una de las categorías en el diccionario calculo 
            # el logaritmo de las probabilidades 'prior'.
            self.log_classPriorProb[c] = math.log(self.classCount[c]/n)
            
            # Diccionario vacio para el conteo de que dada una categoría 
            # se observe una palabra en algún documento.
            self.wordConditionalCounts[c] = {}
            
        # Calculo de los conteos condicionales de cada una de 
        # las palabras ligadas a una categoría C(w|c)
        for text, c in zip(data, clases):# Recorremos dos listas simultaneamente
            # Aplicamos la función conteo que devuelve el número de veces que 
            # aparece el objeto texto en los datos.
            counts = self.word_counts(self.tokenize(text))
            for word, count in counts.items():
                # Vericamos si la palabra no esta en el vocabulario, si no esta, la agregamos
                if word not in self.vocab:
                    self.vocab.add(word)
                # Vericamos si la palabra no esta en los conteos condicionales, si no esta, la agregamos    
                if word not in self.wordConditionalCounts[c]:
                    self.wordConditionalCounts[c][word] = 0.0
                # Hacemos el conteo condicional respectivo    
                self.wordConditionalCounts[c][word] += count

    # Metodo 'predict' donde se hacen prediciones con el modelo entrenado y 
    # debe adminir los datos sobre los cuales debe predecir.
    def predict(self, data):
    results = []
    for text in data:
        words = set(self.tokenize(text))
        scoreProb = {}
        for word in words: 
        if word not in self.vocab: continue #ignoramos palabras nuevas
            
        #suavizado Laplaciano para P(w|c)
        for c in self.unique_clases:
            log_wordClassProb = math.log(
              (self.wordConditionalCounts[c].get(word, 0.0)+1)/(self.classCount[c]+len(self.vocab)))
            scoreProb[c] = scoreProb.get(c, self.log_classPriorProb[c]) + log_wordClassProb
        arg_maxprob = np.argmax(np.array(list(scoreProb.values())))
        results.append(list(scoreProb.keys())[arg_maxprob])
    return results


In [8]:
### Utilidades de Scikit Learn
* `train_test_split`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

* `accuracy_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

* `precision_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

* `recall_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

### Utilidades de Scikit Learn
* `train_test_split`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

* `accuracy_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

* `precision_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

* `recall_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
data_train, data_test, clases_train, clases_test = train_test_split(data, clases, test_size=0.10, random_state=42)

In [ ]:
classifier = NaiveBayesClassifier()
classifier.fit(data_train, clases_train)

In [ ]:
clases_predict = classifier.predict(data_test)

In [ ]:
accuracy_score(clases_test, clases_predict)

In [ ]:
precision_score(clases_test, clases_predict, average=None, zero_division=1)

In [ ]:
recall_score(clases_test, clases_predict, average=None, zero_division=1)

In [ ]:
!jupyter nbconvert --to=python 1_NaiveBayes_class.ipynb

In [ ]:
!jupyter nbconvert --to=python 1_NaiveBayes_class.ipynb